In [72]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import pandas as pd
df_application = pd.read_csv('application_train.csv')
df_application_test = pd.read_csv('application_test.csv')

from pandas_summary import DataFrameSummary
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
# bureau_balance.csv
# bureau.csv
# credit_card_balance.csv
# installments_payments.csv
# POS_CASH_balance.csv
# previous_application.csv
# sample_submission.csv

In [57]:
df_application.head(5)
df_application.shape

(307511, 122)

In [41]:
DataFrameSummary(df_application).summary().T

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
SK_ID_CURR,307511,278181,102790,100002,189146,278202,367142,456255,307511,307511,0,0%,numeric
TARGET,307511,0.0807288,0.272419,0,0,0,0,1,307511,2,0,0%,bool
NAME_CONTRACT_TYPE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307511,2,0,0%,bool
CODE_GENDER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307511,3,0,0%,categorical
FLAG_OWN_CAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307511,2,0,0%,bool
FLAG_OWN_REALTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307511,2,0,0%,bool
CNT_CHILDREN,307511,0.417052,0.722121,0,0,0,1,19,307511,15,0,0%,numeric
AMT_INCOME_TOTAL,307511,168798,237123,25650,112500,147150,202500,1.17e+08,307511,2548,0,0%,numeric
AMT_CREDIT,307511,599026,402491,45000,270000,513531,808650,4.05e+06,307511,5603,0,0%,numeric
AMT_ANNUITY,307499,27108.6,14493.7,1615.5,16524,24903,34596,258026,307499,13672,12,0.00%,numeric


In [58]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0
oh_count = 0
oh_df =pd.DataFrame()
# Iterate through the columns
for col in df_application:
    if df_application[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(df_application[col].unique())) <= 2:
            print('le :' + col)
            # Train on the training data
            le.fit(df_application[col])
            # Transform both training and testing data
            df_application[col] = le.transform(df_application[col])
            #df_application[col] = le.transform(df_application[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
        else:
            print('ce: ' +col)
            oh_df = pd.concat([oh_df,pd.get_dummies(df_application[col],prefix=col+'_')],axis=1)
            df_application.drop([col],axis=1, inplace=True)
                              
print('%d columns were label encoded.' % le_count)

le :NAME_CONTRACT_TYPE
ce: CODE_GENDER
le :FLAG_OWN_CAR
le :FLAG_OWN_REALTY
ce: NAME_TYPE_SUITE
ce: NAME_INCOME_TYPE
ce: NAME_EDUCATION_TYPE
ce: NAME_FAMILY_STATUS
ce: NAME_HOUSING_TYPE
ce: OCCUPATION_TYPE
ce: WEEKDAY_APPR_PROCESS_START
ce: ORGANIZATION_TYPE
ce: FONDKAPREMONT_MODE
ce: HOUSETYPE_MODE
ce: WALLSMATERIAL_MODE
ce: EMERGENCYSTATE_MODE
3 columns were label encoded.


In [59]:
df_application= pd.concat([df_application,oh_df],axis=1)

In [63]:
df_application.shape
df_application.dropna(axis=1,inplace=True)

In [66]:
def compute_score(clf, X, y, scoring='accuracy'):
    #http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py
    xval = cross_val_score(clf, X, y, cv = 10, scoring=scoring)
    return np.mean(xval)

In [67]:
from sklearn.model_selection import train_test_split
X = df_application.drop(['TARGET'],axis=1)
y = df_application['TARGET']
train_features, test_features, train_labels, test_labels = \
train_test_split(X, y, test_size=0.1, random_state=42)

In [70]:

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators = 100,
                                  random_state = 42,
                                  n_jobs=-1,
                                  oob_score=True,
                                  min_samples_split=2,
                                  min_samples_leaf=1,
                                  max_features='auto',
                                  bootstrap=True)
# Train the model on training data
rf_model.fit(train_features, train_labels)


#predictions = rf_model.predict(test_features)
accuracy =  compute_score(rf_model,test_features,test_labels)
print(accuracy)

0.9180866380865748
